# **USE CASE 1.** Comparative study of whether to consider the usage of FL

In this use case we perform a comparative study to decide whether to consider or not the usage of FL in a given scenario.
First, a centralized model is built to consider it as an upper-bound limit of performance. Such upper-bound limit is given by the fact that, when FL is a possibility, the data cannot be usually used in a centralized manner.
Then, local models are built using only their local data and not sharing or communicating with the rest of clients. That would be the case of each client building isolated models without information sharing.
Finally, federated models are built using their local data while not sharing it with any other client, but communicating so a model can be collaboratively built.

This use case is implemented using TensorFlow and TensorFlow Federated (TFF), so the results are not biased by different architectures.

## **Centralized model**

In this first section, it is shown how a centralized model using all available data is built using TensorFlow (which is the base library for TFF). More information about how to build TensorFlow models can be found at [TensorFlow's website](https://www.tensorflow.org/).

In [1]:
import os
import collections
import random
import time

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import models, layers, losses, metrics, optimizers

2023-05-15 17:38:10.223598: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 17:38:10.260850: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 17:38:10.262139: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 17:38:10.898843: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jose/tff/tff_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# Some parameters
NUM_EPOCHS = 50 # Number of epochs (for fair comparison, is the number of epochs * number of rounds in federated scenario)
BATCH_SIZE = 20 # Batch size for training phase
SHUFFLE_BUFFER = 1000 # For shuffling data

# Seeds for random numbers. The experiments are executed 10 times with different seeds
seeds = [10] #, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [3]:
# Method to normalize image data
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

In [4]:
# Method that creates a keras model with a CNN
def create_keras_CNN():
    model = models.Sequential([
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu", padding="same", strides=1),
        layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'),
        layers.Flatten(),
        layers.Dense(10, activation="softmax"),
    ])
                
    return model

In [5]:
# Execute the experiment for each different seed
for seed in seeds:
    # Define the seed for random numbers
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.utils.set_random_seed(seed)

    # Load MNIST dataset from tfds
    (mnist_train, mnist_test), mnist_info = tfds.load('mnist', split=['train', 'test'],
                                                      shuffle_files=True, 
                                                      as_supervised=True, with_info=True)

    # Normalize and prepare train data
    ds_train = mnist_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    example_dataset = next(iter(ds_train))

    ds_train = ds_train.cache()
    ds_train = ds_train.shuffle(mnist_info.splits['train'].num_examples)
    ds_train = ds_train.batch(BATCH_SIZE)
    ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
    
    # Prepare test data
    ds_test = mnist_test.map(
        normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.batch(128)
    ds_test = ds_test.cache()
    ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

    # Create and compile CNN model
    keras_model = create_keras_CNN()

    keras_model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[metrics.SparseCategoricalAccuracy()]
    )

    start_time = time.time()
    
    # Fit the model
    train_metrics = keras_model.fit(
        ds_train, epochs=NUM_EPOCHS, verbose=1,
    )
    train_metrics = train_metrics.history
    print('Training metrics: ')
    print(train_metrics)
    print()

    eval_metrics = keras_model.evaluate(ds_test, verbose=1, return_dict=True)
    print('Test metrics: ')
    print(eval_metrics)
    print()

    end_time = time.time()

    print('Training and testing in {:.2f} seconds'.format(end_time - start_time))

Epoch 1/5


2023-05-15 17:38:12.464470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-05-15 17:38:12.464848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-05-15 17:38:12.492997: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline 

3000/3000 [==============================] - 16s 5ms/step - loss: 0.1599 - sparse_categorical_accuracy: 0.9534
Epoch 2/5
3000/3000 [==============================] - 13s 4ms/step - loss: 0.0593 - sparse_categorical_accuracy: 0.9826
Epoch 3/5
3000/3000 [==============================] - 13s 4ms/step - loss: 0.0425 - sparse_categorical_accuracy: 0.9866
Epoch 4/5
3000/3000 [==============================] - 12s 4ms/step - loss: 0.0325 - sparse_categorical_accuracy: 0.9898
Epoch 5/5
3000/3000 [==============================] - 13s 4ms/step - loss: 0.0249 - sparse_categorical_accuracy: 0.9922
Training metrics: 
{'loss': [0.15989036858081818, 0.05932208150625229, 0.042547017335891724, 0.03249415010213852, 0.02490651048719883], 'sparse_categorical_accuracy': [0.9533500075340271, 0.9825666546821594, 0.9865666627883911, 0.9897500276565552, 0.9921666383743286]}

 8/79 [==>...........................] - ETA: 0s - loss: 0.0633 - sparse_categorical_accuracy: 0.9795 

2023-05-15 17:39:19.114978: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]
2023-05-15 17:39:19.115439: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]


79/79 [==============================] - 1s 7ms/step - loss: 0.0517 - sparse_categorical_accuracy: 0.9821
Test metrics: 
{'loss': 0.051694318652153015, 'sparse_categorical_accuracy': 0.9821000099182129}

Training and testing in 67.28 seconds


## **Local models**

In this section, we build local models using only their local data. The models are evaluated using a global test set, so all models are evaluated over the same data, which is also IID or non-IID, according to the characteristics of the data. Specifically, such global test set is comprised by all the client's test sets (the same procedure is also follow for the federated models, for a fair comparison). In this case, the models are also created in TensorFlow; more information about TensorFlow can be found at [TensorFlow's website](https://www.tensorflow.org/). TFF is used in this scenario to load the non-IID data if neccesary.

In [32]:
import os
import collections
import random
import time

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds

from tensorflow_federated.python.simulation.datasets import emnist
from tensorflow_federated.python.learning.algorithms import build_unweighted_fed_avg, build_fed_eval
from tensorflow.keras import models, layers, losses, metrics, optimizers

In [33]:
# Some parameters
NUM_CLIENTS = 10 # Number of clients
NUM_EPOCHS = 50 # Number of epochs (for fair comparison, is the number of epochs * number of rounds in federated scenario)
BATCH_SIZE = 20 # Batch size for training phase

In [34]:
# Indicates if IID data is used; if False, non-IID partitions are used
is_iid = False

# Seeds for random numbers. Execute the experiment several times.
seeds = [10] #, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [35]:
# Normalize the images
def normalize_img(element):
    """Normalizes images: `uint8` -> `float32`."""
    if is_iid:
        image, label = element['image'], element['label']
        image = tf.cast(image, tf.float32) / 255.
    else:
        image, label = element['pixels'], element['label']
        image = tf.cast(image, tf.float32)
    return image, label

def normalize_test_niid(element):
    image, label = element['image'], element['label']
    image = tf.cast(image, tf.float32) / 255.
    return image, label

In [36]:
# Preprocess the data
def preprocess(dataset):
    return dataset.shuffle(100, seed=seed).batch(BATCH_SIZE).map(normalize_img).prefetch(tf.data.AUTOTUNE)

In [37]:
# This method receives a client_id, and returns the training tf.data.Dataset for that client
def create_tf_dataset_for_client_fn_train(client_id):
    client_data = mnist_train_df[mnist_train_df['id'] == client_id].drop(columns='id')
    return tf.data.Dataset.from_tensor_slices(client_data.to_dict('list'))

# This method receives a client_id, and returns the testing tf.data.Dataset for that client
def create_tf_dataset_for_client_fn_test(client_id):
    client_data = mnist_test_df[mnist_test_df['id'] == client_id].drop(columns='id')
    return tf.data.Dataset.from_tensor_slices(client_data.to_dict('list'))

# Create a list of datasets (one for each client) from the complete dataset and the number of 
# clients (it will select the first client ids for simulation).
def make_federated_data(client_data, n_clients):    
    return [
        preprocess(client_data.create_tf_dataset_for_client(x)) # Call previous preprocess method
        for x in client_data.client_ids[0:n_clients]
    ]

In [38]:
# Method that creates a keras model with a CNN
def create_keras_CNN():
    model = models.Sequential([
        layers.Reshape((28, 28, 1), input_shape=(28, 28)),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu", padding="same", strides=1),
        layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'),
        layers.Flatten(),
        layers.Dense(10, activation="softmax"),
    ])
                
    return model

In [39]:
# Run experiment for each different seed
for seed in seeds:
    # Define the seed for random numbers
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.utils.set_random_seed(seed)

    # Load either IID or non-IID data
    if not is_iid:
        # Load federated version of mnist from TFF (== EMNIST loading only the digits)
        mnist_train, mnist_test = emnist.load_data(only_digits=True)
    else:
        # Load MNIST from tfds, and get train and test partitions
        mnist = tfds.load('mnist')
        mnist_train, mnist_test = mnist['train'], mnist['test']

        # Transform the data to a dataframe
        mnist_train_df = tfds.as_dataframe(mnist_train)

        # Create a random list of ids. Each instance is given a random id meaning the client where will be distributed
        ids_train = [i for i in range(NUM_CLIENTS) for _ in range(len(mnist_train)//NUM_CLIENTS)]
        random.Random(seed).shuffle(ids_train)
        # Add the id assignment to the dataframe
        mnist_train_df['id'] = ids_train

        # Do the same with the test data
        mnist_test_df = tfds.as_dataframe(mnist_test)
        ids_test = [i for i in range(NUM_CLIENTS) for _ in range(len(mnist_test)//NUM_CLIENTS)]
        random.Random(seed+1).shuffle(ids_test)
        mnist_test_df['id'] = ids_test

        # Get traning and testing datasets for each client
        mnist_train = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
            client_ids=list(range(0,NUM_CLIENTS)),
            serializable_dataset_fn=create_tf_dataset_for_client_fn_train
        )
        mnist_test = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
            client_ids=list(range(0,NUM_CLIENTS)),
            serializable_dataset_fn=create_tf_dataset_for_client_fn_test
        )

    # Create the federated train data from the full mnist_train data, and filtering only 
    # NUM_CLIENTS clients
    train_data = make_federated_data(mnist_train, NUM_CLIENTS)
    test_data = make_federated_data(mnist_test, NUM_CLIENTS)

    # Build model for each client
    for client in range(NUM_CLIENTS):
        keras_model = create_keras_CNN()

        keras_model.compile(
            optimizer=optimizers.Adam(learning_rate=0.001),
            loss=losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=[metrics.SparseCategoricalAccuracy()]
        )

        start_time = time.time()
            
        # Train model
        train_metrics = keras_model.fit(
            train_data[client], epochs=NUM_EPOCHS, verbose=1,
        )
        
        train_metrics = train_metrics.history
        print('Training metrics: ')
        print(train_metrics)
        print()
        
        # Evaluate the model over global test data
        # i.e., for ease of simulation, evaluate over each client's test set and average
        test_loss = 0
        test_acc = 0
        for test_client in range(NUM_CLIENTS):
            metrics_test = keras_model.evaluate(test_data[test_client], verbose=1,
                                                return_dict=True)
            test_loss += metrics_test['loss']
            test_acc += metrics_test['sparse_categorical_accuracy']
                                                
        test_loss /= NUM_CLIENTS
        test_acc /= NUM_CLIENTS
                                                
        print('Clients test matrics: ')
        print(f"loss: {test_loss}\t sparse_categorical_accuracy: {test_acc}")
        
        end_time = time.time()
    
        print('Training and testing in {:.2f} seconds'.format(end_time - start_time))

Epoch 1/50
5/5 [==============================] - 0s 9ms/step - loss: 2.4450 - sparse_categorical_accuracy: 0.1075
Epoch 2/50
5/5 [==============================] - 0s 7ms/step - loss: 2.3082 - sparse_categorical_accuracy: 0.1398
Epoch 3/50
5/5 [==============================] - 0s 8ms/step - loss: 2.2953 - sparse_categorical_accuracy: 0.0860
Epoch 4/50
5/5 [==============================] - 0s 9ms/step - loss: 2.2325 - sparse_categorical_accuracy: 0.1505
Epoch 5/50
5/5 [==============================] - 0s 16ms/step - loss: 2.1920 - sparse_categorical_accuracy: 0.5054
Epoch 6/50
5/5 [==============================] - 0s 12ms/step - loss: 2.1466 - sparse_categorical_accuracy: 0.3548
Epoch 7/50
5/5 [==============================] - 0s 7ms/step - loss: 2.0892 - sparse_categorical_accuracy: 0.4839
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - loss: 2.0241 - sparse_categorical_accuracy: 0.6667
Epoch 9/50
5/5 [==============================] - 0s 7ms/step - loss: 1.9635 -

## **Federated models**

In this section we build a shared global model between all clients using FL. Each client updates the model using its local data and send the updates to the central server, that orchestrates the global learning. All the models are evaluated simulating a global test set, comprised of each client's test data (as previously done with the local models). More detailed information and analysis about how to build FL models is provided in subsequent use cases.

In [40]:
import collections
import random
import time
import os

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds

from tensorflow_federated.python.simulation.datasets import emnist
from tensorflow_federated.python.learning.algorithms import build_unweighted_fed_avg, build_fed_eval
from tensorflow.keras import models, layers, losses, metrics, optimizers

from tensorflow_federated.python.learning.model_update_aggregator import dp_aggregator

In [41]:
# Some parameters
NUM_CLIENTS = 10 # Number of clients in the federated scenario
NUM_ROUNDS = 10 # Number of learning rounds in the federated computation
NUM_EPOCHS = 5 # Number of epochs that the local dataset is seen each round
BATCH_SIZE = 20 # Batch size for training phase

In [42]:
# Indicates if IID data is used; if False, non-IID partitions are used
is_iid = False

# Seeds for random numbers. Execute the experiment several times.
seeds = [10] #, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [43]:
# Preprocess federated data
def preprocess(dataset):
    def batch_format_fn(element):
        if is_iid:
            return collections.OrderedDict(
                x=element['image']/255,
                y=element['label']
            )
        else:
            return collections.OrderedDict(
                x=element['pixels'],
                y=element['label']
            )

    return dataset.repeat(NUM_EPOCHS).shuffle(100, seed=seed).batch(BATCH_SIZE).map(batch_format_fn)


# Construct a list of datasets (one for each client) from the complete dataset and the number of 
# clients (it will select the first client ids for simulation).
def make_federated_data(client_data, n_clients): 
    return [
        preprocess(client_data.create_tf_dataset_for_client(x)) # Call previous preprocess method
        for x in client_data.client_ids[0:n_clients]
    ]


# Create testing federated dataset
# If t is none, each client receives its testing data
# If t is a number, the t-th testing data is given to all clients
def make_federated_data_test(client_data, n_clients, t=None): 
    if t is None:
        # Return the test data of each client
        return [
            preprocess(client_data.create_tf_dataset_for_client(x)) 
            for x in client_data.client_ids[0:n_clients]
        ]
    else:
        # Return the test data of a given client t
        return [
            preprocess(client_data.create_tf_dataset_for_client(x)) 
            for x in [client_data.client_ids[t]]*n_clients
        ]

In [44]:
# Method that creates a keras model with a CNN
def create_keras_CNN():
    model = models.Sequential([
        layers.Reshape((28, 28, 1), input_shape=(28, 28)),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu", padding="same", strides=1),
        layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'),
        layers.Flatten(),
        layers.Dense(10, activation="softmax"),
    ])
        
    return model


def model_fn():
    keras_model = create_keras_CNN()
    
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=train_data[0].element_spec,
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=[metrics.SparseCategoricalAccuracy()]
    )

In [45]:
# Run experiment for each different seed
for seed in seeds:
    # Define the seed for random numbers
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.utils.set_random_seed(seed)
    
    # Load either IID or non-IID data
    if not is_iid:
        # Load federated version of mnist from TFF (== EMNIST loading only the digits)
        mnist_train, mnist_test = emnist.load_data(only_digits=True)
    else:
        # Load MNIST from tfds, and get train and test partitions
        mnist = tfds.load('mnist')
        mnist_train, mnist_test = mnist['train'], mnist['test']

        # Transform the data to a dataframe
        mnist_train_df = tfds.as_dataframe(mnist_train)

        # Create a random list of ids. Each instance is given a random id meaning the client where will be distributed
        ids_train = [i for i in range(NUM_CLIENTS) for _ in range(len(mnist_train)//NUM_CLIENTS)]
        random.Random(seed).shuffle(ids_train)
        # Add the id assignment to the dataframe
        mnist_train_df['id'] = ids_train

        # Do the same with the test data
        mnist_test_df = tfds.as_dataframe(mnist_test)
        ids_test = [i for i in range(NUM_CLIENTS) for _ in range(len(mnist_test)//NUM_CLIENTS)]
        random.Random(seed+1).shuffle(ids_test)
        mnist_test_df['id'] = ids_test

        # Get traning datasets for each client
        mnist_train = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
            client_ids=list(range(0,NUM_CLIENTS)),
            serializable_dataset_fn=create_tf_dataset_for_client_fn_train
        )

    # Create the federated train data from the full mnist_train data, and filtering only 
    # NUM_CLIENTS clients
    train_data = make_federated_data(mnist_train, NUM_CLIENTS)
    
    start_time = time.time()

    # Define training strategy
    training_process = build_unweighted_fed_avg(
        model_fn,
        client_optimizer_fn=lambda: optimizers.Adam(learning_rate=0.001),
        server_optimizer_fn=lambda: optimizers.Adam(learning_rate=0.01),
    )

    # Train the model
    train_state = training_process.initialize()

    for round_num in range(1, NUM_ROUNDS+1):
        # Train next round (send model to clients, local training, and server model averaging)
        result = training_process.next(train_state, train_data)

        # Current state of the model
        train_state = result.state

        # Get and print metrics, as the loss and accuracy (averaged across all clients)
        train_metrics = result.metrics['client_work']['train']

        if round_num == NUM_ROUNDS:
            print('Round {:2d},  \t Loss={:.4f}, \t Accuracy={:.4f}'.format(round_num, train_metrics['loss'],
                                                                            train_metrics['sparse_categorical_accuracy']))

    # Indicate that the model arquitecture is the one proposed before
    evaluation_process = build_fed_eval(model_fn)

    # Initialize the process and set the weights to those previously trained (getting from the training
    # state and setting to the evaluation one).
    evaluation_state = evaluation_process.initialize()
    model_weights = training_process.get_model_weights(train_state)
    evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

    # Average loss and accuracy
    losses_clients = []
    accuracies_clients = []
    
     # Evaluate on the test set of each client
    for i in range(NUM_CLIENTS):
        test_data = make_federated_data_test(mnist_test, NUM_CLIENTS, t=i) # data of i-th client

        # Pass test data to the model in each client
        evaluation_output = evaluation_process.next(evaluation_state, test_data)

        # Get and print metrics
        eval_metrics = evaluation_output.metrics['client_work']['eval']['current_round_metrics']
        print('Test data, \t Loss={:.4f}, \t Accuracy={:.4f}'.format(eval_metrics['loss'], 
                                                                     eval_metrics['sparse_categorical_accuracy']))

        losses_clients.append(eval_metrics['loss'])
        accuracies_clients.append(eval_metrics['sparse_categorical_accuracy'])

    # Print mean metrics over all test sets
    print()
    print('Mean test (global), \t Loss={:.4f}, \t Accuracy={:.4f}'.format(np.mean(losses_clients), np.mean(accuracies_clients)))

    end_time = time.time()
    
    print('Training and testing in {:.2f} seconds'.format(end_time - start_time))

2023-05-15 18:11:34.569103: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'batch_input' with dtype float and shape [?,28,28]
	 [[{{node batch_input}}]]
2023-05-15 18:11:34.651189: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_1' with dtype float and shape [?,28,28]
	 [[{{node args_1}}]]
2023-05-15 18:11:34.704241: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_3' w

Round 10,  	 Loss=0.7103, 	 Accuracy=0.8527


2023-05-15 18:11:48.792258: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'batch_input' with dtype float and shape [?,28,28]
	 [[{{node batch_input}}]]
2023-05-15 18:11:48.865417: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_1' with dtype float and shape [?,28,28]
	 [[{{node args_1}}]]
2023-05-15 18:11:49.148172: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-05-15 18:11:49.148243: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-15 18:11:49.161431: I tensorflow/core/grappler/devices.cc:66] Number of 

Test data, 	 Loss=1.8248, 	 Accuracy=0.3636
Test data, 	 Loss=1.0208, 	 Accuracy=0.6154
Test data, 	 Loss=0.9575, 	 Accuracy=0.6667
Test data, 	 Loss=1.5943, 	 Accuracy=0.5000
Test data, 	 Loss=0.8071, 	 Accuracy=0.6667
Test data, 	 Loss=0.8005, 	 Accuracy=0.6667
Test data, 	 Loss=0.7980, 	 Accuracy=0.6154
Test data, 	 Loss=0.6137, 	 Accuracy=0.9000
Test data, 	 Loss=1.7456, 	 Accuracy=0.3077
Test data, 	 Loss=1.4729, 	 Accuracy=0.4545

Mean test (global), 	 Loss=1.1635, 	 Accuracy=0.5757
Training and testing in 18.51 seconds
